In [134]:
import sys
import os
import pandas as pd
import re 
from langchain_community.document_loaders import PyMuPDFLoader

# Dynamically resolve the base directory for Jupyter notebooks
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '../F1-regulation-RAG-2025'))
sys.path.append(BASE_DIR)

In [135]:
# Ensure the data directory path is resolved correctly
processed_data_dir = os.path.join(BASE_DIR, 'data', 'processed')
os.makedirs(processed_data_dir, exist_ok=True)
processed_data_dir 

'/Users/pierre.muller/Documents/Perso/F1-regulation-RAG-2025/data/processed'

In [136]:
!ls {BASE_DIR}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


README.md      examples       pyproject.toml uv.lock
data           main.py        src


In [137]:
def remove_footer(text: str) -> str:
    """
    Removes the footer from the given text.

    Args:
        text (str): The text containing the footer.

    Returns:
        str: The text with the footer removed.
    """
    footer_pattern = r"2025 Formula 1 Sporting Regulations\s+\d+/\d+\s+30 April 2025\s+©2025 Fédération Internationale de l’Automobile\s+Issue 5"
    cleaned_text = re.sub(footer_pattern, '', text)
    return cleaned_text

In [138]:
def extract_pdf_text_to_dataframe(pdf_path: str) -> pd.DataFrame:
    """
    Extracts text from a PDF file and returns it in a pandas DataFrame.

    Args:
        pdf_path (str): Path to the PDF file.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted text, with each page as a row.
    """
    try:
        
        # Load the document
        loader = PyMuPDFLoader(pdf_path,extract_tables='markdown')
        documents = loader.load_and_split()

        pages_text = [remove_footer(doc.page_content) for doc in documents]

        df = pd.DataFrame({'Page': range(1, len(pages_text) + 1), 'Text': pages_text})
        return df


    except Exception as e:
        raise RuntimeError(f"Error processing PDF file: {e}")



In [139]:
pdf_path = os.path.join(BASE_DIR, 'data', 'pdfs', 'FIA 2025 Formula 1 Sporting Regulations - Issue 5 - 2025-04-30.pdf')
df = extract_pdf_text_to_dataframe(pdf_path)
df.to_csv(os.path.join(processed_data_dir, 'pdf_text.csv'), index=False)
print(df['Text'][1])

 
 
1) 
REGULATIONS 
1.1 
The FIA will organise the FIA Formula One World Championship (the Championship) which is the 
property of the FIA and comprises two titles of World Champion, one for drivers and one for 
constructors. It consists of the Formula One Grand Prix races which are included in the Formula 
One calendar and in respect of which the ASNs and organisers have signed organisation 
agreements with the FIA. All the participating parties (FIA, ASNs, organisers, Competitors and 
circuits) undertake to apply as well as observe the rules governing the Championship and must 
hold the appropriate FIA Licences which are issued to drivers, Competitors, officials, organisers 
and circuits. 
1.2 
The final text of these Sporting Regulations shall be the English version which will be used should 
any dispute arise as to their interpretation. Headings in this document are for ease of reference 
only and do not form part of these Sporting Regulations. 
1.3 
These Sporting Regulations app

In [140]:
df.iloc[-1]['Text']

' \n \nAPPENDIX 9 \n \nAPPROVED CHANGES FOR SUBSEQUENT YEARS \n \nConvention: \nDark red text: All changes previously agreed for subsequent years \nDark red highlighted (yellow) text: new changes for subsequent years, approved by the WMSC \n \n1 \nChanges for 2026'

In [141]:
match = re.search(r"APPENDIX[\s\n]+\d+", df.iloc[-1]['Text'])
if match:
    print("Match found:", match.group())
else:
    print("No match found.")

Match found: APPENDIX 9


In [142]:
def extract_sections_to_dataframe(text: str) -> pd.DataFrame:
    """
    Extracts sections from the text based on patterns like 'NUMBER) A TITLE'
    and returns a DataFrame with 'title' and 'chunk' columns.

    Args:
        text (str): The text to process.

    Returns:
        pd.DataFrame: A DataFrame with 'title' and 'chunk' columns.
    """
    # Updated regex to handle cases like "VIRTUAL SAFETY CAR (VSC)"
    # and exclude titles with letters before the number
    section_pattern = r"(?<![A-Z])(\d+\)\s+(?:[A-Z ,\n\-\(\)É]+))\n"
    matches = re.split(section_pattern, text)

    titles = []
    chunks = []

    for i in range(1, len(matches), 2):
        titles.append(matches[i].replace('\n', ' ').strip())  # Clean multi-line titles
        chunks.append(matches[i + 1].strip())

    processed_df = pd.DataFrame({'title': titles, 'chunk': chunks})
    processed_df[['title', 'chunk']] = processed_df[['title', 'chunk']].map(lambda x: x.replace('\n', ' ').strip())

    return processed_df

# Process the extracted text into sections
df_sections = pd.concat([extract_sections_to_dataframe(page) for page in df['Text']], ignore_index=True)
df_sections.to_csv(os.path.join(processed_data_dir, 'pdf_sections.csv'), index=False)
print(df_sections.tail())

                                                title  \
59          60)  POST SPRINT AND POST RACE PARC FERMÉ   
60                 61)  SPRINT SESSION CLASSIFICATION   
61                           62)  RACE CLASSIFICATION   
62  63)  PODIUM CEREMONY AND POST RACE PRESS CONFE...   
63                                64)  TEAM EQUIPMENT   

                                                chunk  
59  60.1  Only those officials charged with superv...  
60  61.1  The car placed first will be the one hav...  
61  62.1  The car placed first will be the one hav...  
62  63.1  The drivers finishing the race in 1st, 2...  
63  64.1  All equipment used to cool the car on th...  


In [119]:
# increase pandas display max rows 100
pd.set_option('display.max_rows', 100)
len(df_sections)

64

# Appendices must be added

In [ ]:
def extract_appendices_to_dataframe(text: str) -> pd.DataFrame:
    """
    Extracts sections from the text based on patterns like 'NUMBER) A TITLE'
    and returns a DataFrame with 'title' and 'chunk' columns.

    Args:
        text (str): The text to process.

    Returns:
        pd.DataFrame: A DataFrame with 'title' and 'chunk' columns.
    """
    section_pattern = r"APPENDIX[\s\n]+\d+"
    matches = re.split(section_pattern, text)
    print(matches)

    titles = []
    chunks = []

    for i in range(1, len(matches), 2):
        titles.append(matches[i].strip())
        print(f"Title {i//2 + 1}: {matches[i].strip()}")
        chunks.append(matches[i + 1].strip())

    return pd.DataFrame({'title': titles, 'chunk': chunks})

# Process the extracted text into sections
df_appendices = pd.concat([extract_appendices_to_dataframe(page) for page in df['Text']], ignore_index=True)
df_appendices.to_csv(os.path.join(processed_data_dir, 'pdf_appendices.csv'), index=False)
print(df_appendices.tail())

title                                 64) \nTEAM EQUIPMENT
chunk    64.1 \nAll equipment used to cool the car on t...
Name: 56, dtype: object

### Token embeddings

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")


In [122]:
df = pd.read_csv(os.path.join(processed_data_dir, 'pdf_sections.csv'))

In [123]:
df.head()  # Display the first few rows of the DataFrame

,title,chunk
0,1) REGULATIONS,1.1 The FIA will organise the FIA Formula One...
1,2) GENERAL UNDERTAKING,"2.1 All drivers, Competitors and officials pa..."
2,3) GENERAL CONDITIONS,3.1 It is the Competitor’s responsibility to ...
3,4) LICENCES,"4.1 All drivers, Competitors and officials pa..."
4,5) CHAMPIONSHIP COMPETITIONS,5.1 Competitions are reserved for Formula One...


In [124]:
df.loc[df.isna().sum(axis=1) > 0]

,title,chunk
28,29) VOID,NaN
40,41) VOID,NaN


In [125]:
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")
embeddings = model.encode(df['chunk'].tolist(), show_progress_bar=True, device='mps')

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [126]:
embeddings

array([[ 0.03867137, -0.00604724, -0.00682677, ..., -0.03376015,
        -0.00947953, -0.00209979],
       [ 0.0311457 , -0.03375433, -0.00729547, ..., -0.04132152,
        -0.00200754, -0.02746743],
       [ 0.05352963, -0.03765709, -0.00557554, ..., -0.01629589,
        -0.02041579,  0.00070395],
       ...,
       [ 0.00495611, -0.02649595, -0.00973289, ..., -0.01721682,
         0.00948874, -0.04745426],
       [ 0.04810619, -0.0259371 , -0.00585549, ...,  0.02249218,
         0.00224992, -0.00090425],
       [ 0.03289371,  0.01738825, -0.00715252, ..., -0.04115681,
        -0.03080961, -0.00738988]], shape=(64, 1024), dtype=float32)

In [130]:
model.similarity(model.encode("What is the VSC?"), embeddings)

tensor([[0.1843, 0.1749, 0.2323, 0.2085, 0.2418, 0.2401, 0.3081, 0.1699, 0.1859,
         0.2654, 0.2676, 0.2892, 0.1821, 0.2115, 0.2099, 0.2381, 0.2024, 0.1846,
         0.1996, 0.2168, 0.2357, 0.2096, 0.1788, 0.3229, 0.2023, 0.2564, 0.1876,
         0.2377, 0.5478, 0.1695, 0.2415, 0.1461, 0.2723, 0.1519, 0.1893, 0.2173,
         0.2170, 0.2490, 0.2368, 0.2996, 0.5478, 0.1999, 0.3463, 0.1779, 0.2154,
         0.2779, 0.1876, 0.2692, 0.2043, 0.2816, 0.3420, 0.2849, 0.2743, 0.2674,
         0.2068, 0.5777, 0.2131, 0.2295, 0.2824, 0.2243, 0.2554, 0.2894, 0.1919,
         0.3097]])

In [131]:
model.similarity(model.encode("What is the VSC?"), embeddings).argmax()

tensor(55)

In [132]:
df['chunk'][55]

'56.1  The VSC procedure may be initiated to neutralise a practice session, sprint session or a race upon  the order of the clerk of the course.  a)  It will normally be used when double waved yellow flags are needed on any section of  track and Competitors or officials may be in danger, but the circumstances are not such  as to warrant use of the safety car itself.  56.2  When the order is given to initiate the VSC procedure a message “VSC DEPLOYED” will be sent  to all Competitors using the official messaging system and all FIA light panels will display “VSC”.  56.3  No car may be driven unnecessarily slowly, erratically or in a manner which could be deemed  potentially dangerous to other drivers or any other person at any time whilst the VSC procedure  is in use. This will apply whether any such car is being driven on the track, the pit entry road, or  the pit lane.  56.4  When initiated during a sprint session or a race, no car may enter the pits whilst the VSC  procedure is in use